In [2]:
#Run the optimal kmeans to remove the benign cluster from data (without capture 9)
import pandas as pd
import os
from sklearn.cluster import KMeans
import numpy as np

In [3]:

#load data
os.chdir('../..') #to change working directory of the notebook when restarted
#Initialize empty df
df_all=pd.DataFrame()
for root, dirs, files in os.walk("./data/csv/captures_features"):
    for ctuName in files:
        #ignore capture 9
        if ctuName == 'capture20110817.binetflow.labels-positive-weights.labeled.csv':
            continue
        df=pd.read_csv("./data/csv/captures_features/"+ctuName)
        #concat to general df as new rows
        df_all=pd.concat([df_all,df],axis=0,ignore_index=True)
#Change background and normal to 0 and infected values to 1
df_all['label']=df_all['label'].replace(['background','normal'],0)
df_all['label']=df_all['label'].replace(['infected'],1)
#drop node column
df_all=df_all.drop(['node'], axis=1)


df_all

,ID,OD,IDW,ODW,BC,LCC,AC,label
0,0,1,0,1639,0.000000e+00,0.0,1.000000,0
1,6,0,2113,0,0.000000e+00,0.0,16.895299,0
2,1,1,2,2,0.000000e+00,0.0,0.997643,0
3,135476,140755,1397778,1398024,3.080584e+10,0.0,-0.117842,0
4,1,1,7,7,0.000000e+00,0.0,0.999671,0
...,...,...,...,...,...,...,...,...
2874208,0,1,0,3,0.000000e+00,0.0,1.000000,0
2874209,0,1,0,6,0.000000e+00,0.0,1.000000,0
2874210,3,0,24,0,0.000000e+00,0.0,1.240000,0
2874211,0,1,0,6,0.000000e+00,0.0,1.000000,0


In [4]:
#Get dataframe column labels
features=df_all.columns.values
X = np.array(df_all[features[1:8]])
#y = np.array(df_all[features[8]])

#Run kmeans
kmeans = KMeans(n_clusters=100, random_state=1,n_init=1).fit(X)
cluster_labels = kmeans.predict(X)
centroids=kmeans.cluster_centers_
centroids_df=pd.DataFrame(centroids,columns=features[1:8])

In [5]:
df_all['cluster'] = cluster_labels
print(df_all['cluster'].value_counts())

0     2857710
85       4559
63       3152
43       2132
73       1266
       ...   
82          1
83          1
57          1
19          1
15          1
Name: cluster, Length: 100, dtype: int64


In [6]:
df_all['cluster'] = cluster_labels
#drop data inside the benign cluster
#drop most common cluster
benign_str=df_all['cluster'].value_counts().idxmax()
phase2_data=df_all[df_all['cluster']!=benign_str]
#drop Cluster column
phase2_data.drop(['cluster'], axis=1, inplace=True)
phase2_data

/tmp/ipykernel_118098/1587811879.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phase2_data.drop(['cluster'], axis=1, inplace=True)


,ID,OD,IDW,ODW,BC,LCC,AC,label
3,135476,140755,1397778,1398024,3.080584e+10,0.00000,-0.117842,0
5,651,664,194078,160979,1.462356e+08,0.00000,-0.004701,0
7,19269,19665,83768,78421,5.721328e+09,0.00000,-0.822473,0
53,2,2,2,2,4.699049e+06,0.00000,0.990597,0
55,3115,2690,62309,46538,8.260494e+08,0.00000,-0.175081,0
...,...,...,...,...,...,...,...,...
2862464,3,3,3,3,1.023093e+06,0.00000,1.109136,0
2862476,5,4,64,63,2.919229e+06,0.00000,5.359655,0
2862485,4,3,55,3,1.023093e+06,0.62069,-1.701260,0
2862491,3,3,5,5,1.212971e+06,0.00000,1.205473,0


In [7]:
#check hob and bob te verify right run
phase2_data['label'].value_counts()

0    16481
1       22
Name: label, dtype: int64

In [8]:
#save data to csv
phase2_data.to_csv('./data/csv/phase2_data.csv', index=False)
centroids_df.to_csv('./data/csv/phase1_centroids.csv', index=False)
